# Bonito Train
![](https://github.com/nanoporetech/ont_tutorial_transcriptome/raw/master/Static/Images/ONT_logo.png)

This notebook demonstrates the process of training a [basecaller](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1727-y) for [Oxford Nanopore reads](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-018-1462-9), following the method demonstrated in [bonito](https://github.com/nanoporetech/bonito). This basecaller leverages NVIDIA's recently published [QuartzNet](https://arxiv.org/pdf/1910.10261.pdf) convolutional neural network architecture.

A modestly sized dataset is provided however it is not anticipated that models trained using these data exclusively will be production quality. The notbook serves as a learning resource only.


#### Install/import packages

In [0]:
#@title
!pip install -q ont-bonito
!pip install -U -q PyDrive

import os
import sys
import time
import random
from datetime import datetime
from itertools import starmap

from google.colab import auth
from google.colab import drive as gdrive
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

import numpy as np
import pandas as pd
import toml
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

from bonito.model import Model
from bonito.util import accuracy
from bonito.training import ChunkDataSet
from bonito.decode import decode, decode_ref

#### Set up data import functions

In [0]:
# Authenticate and create PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# helper functions for importing data
def download_npy_from_link(fn, link):
    _, id = link.split('=')
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile(fn)
    return np.load(fn)

def download_toml_from_link(fn, link):
    _, id = link.split('=')
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile(fn)
    return toml.load(fn)

#### Specify training data
Required data for 100,000 signal 'chunks' will be loaded from a Google Drive in the form of 4 numpy files.
- chunks.npy
- chunk_lengths.npy
- references.npy
- reference_lengths.npy

If you have an alternative set of four matching files in a Google Drive, you can overwrite the sharable links below. See [here](https://github.com/nanoporetech/bonito/issues/4) for a description of those files).

In [0]:
chunks_link = "https://drive.google.com/open?id=1aciNfQs53eFRwnMggInY-Uisi-owtmzY" #@param {type:"string"}
chunk_lengths_link = "https://drive.google.com/open?id=1sW31y1IdkqjyWb9WNSun7iLRznJQ3hfs" #@param {type:"string"}
references_link = "https://drive.google.com/open?id=1kcs_hZMndUIDX2n8dTxGrAgCvt_TpUcH" #@param {type:"string"}
reference_lengths_link = "https://drive.google.com/open?id=1-r7XymddP_3gKFb-7ohB_t14u7u4SGLm" #@param {type:"string"}
quartznet_config_link = "https://drive.google.com/open?id=1mLqxHMYKA4vfK9wd_2YgaBGzWPVBeilI" #@param {type:"string"}

#### Import training data

In [0]:
#@title

print('Loading chunks.')
full_chunks = download_npy_from_link('chunks.npy',
                                chunks_link)
# Sections of squiggle that correspond with the target reference sequence
# Variable length and zero padded (upto 4096 samples).
# shape (1000000, 4096)
# dtype('float32')

print('Loading chunk lengths.')
full_chunk_lengths = download_npy_from_link('chunk_lengths.npy',
                                       chunk_lengths_link)
# Lengths of squiggle sections in chunks.npy 
# shape (1000000,)
# dtype('uint16')

print('Loading references.')
full_targets = download_npy_from_link('references.npy',
                                 references_link)
# Integer encoded target sequence {'A': 1, 'C': 2, 'G': 3, 'T': 4}
# Variable length and zero padded (default range between 128 and 256).
# shape (1000000, 256)
# dtype('uint8')

print('Loading reference lengths.')
full_target_lengths = download_npy_from_link('reference_lengths.npy',
                                        reference_lengths_link)
# Lengths of target sequences in references.npy
# shape (1000000,)
# dtype('uint8')

print('Loading quartznet config.')
quartznet_config = download_toml_from_link("config_quartznet5x5.toml",
                                           quartznet_config_link)
# The structure of the model is defined using a config file.
# This will make sense to those familar with QuartzNet
# https://arxiv.org/pdf/1910.10261.pdf).

#### Training options
Default options are set, and ranges are sensible, but most combinations of settings are untested.

The default settings will train on a small amount of data (1000 signal chunks) for a small number of epochs (20). This is unlikely to produce an accurate generalisable model, but will train relatively quickly.

After modifying this cell, Runtime -> Run after, so that all cells between this one and the main train looping will be run in accordance with new setting.

A train_proportion of 0.90 will use 90% of the data for training and 10% for validation.

No dropout is applied by default, but in order to avoid overfitting on small data sets it may be necessary to apply dropout (e.g. of 0.5), or other regularisation techniques.

In [0]:
model_savepath = '/content/drive/My Drive/nitobook/' #@param {type:"string"}
learning_rate = 0.001 #@param {type:"number"}
random_seed = 25 #@param {type:"integer"}
epochs = 20 #@param {type:"slider", min:1, max:1000, step:1}
batch_size = 16 #@param [2, 4, 8, 16, 28] {type:"raw"}
num_chunks = 10000 #@param [10, 100, 1000, 10000, 100000] {type:"raw"}
train_proportion = 0.80 #@param type:"slider", min:0.8, max:1000, step:1
dropout = 0.0 #@param {type:"slider", min:0.0, max:0.8}

In [0]:
def init(random_seed):
    """Initialise random libs and setup cudnn
 
    https://pytorch.org/docs/stable/notes/randomness.html
    """
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# set random seeds and setup pytorch
init(random_seed)

# we exploit GPU for training
device = torch.device("cuda")

#### Prepare data according to values set in the 'Training options'

In [0]:
# subset
chunks = full_chunks[:num_chunks]
chunk_lengths = full_chunk_lengths[:num_chunks]
targets = full_targets[:num_chunks]
target_lengths = full_target_lengths[:num_chunks]

# shuffle
shuf = np.random.permutation(chunks.shape[0])
chunks = chunks[shuf]
chunk_lengths = chunk_lengths[shuf]
targets = targets[shuf]
target_lengths = target_lengths[shuf]

split = np.floor(chunks.shape[0] * train_proportion).astype(np.int32)

Modify quartznet config to set dropout.

In [0]:
for b in quartznet_config['block']:
    b['dropout'] = dropout
quartznet_config

#### Train and test functions

In [0]:

# 'Connectionist Temporal Classification' (CTC) loss fuction
# https://distill.pub/2017/ctc/
criterion = nn.CTCLoss(reduction='mean')

def train(log_interval, model, device, train_loader,
          optimizer, epoch, use_amp=False):

    t0 = time.perf_counter()
    chunks = 0

    model.train()

    sys.stderr.write("\n" + "Training epoch: " + str(epoch) + "\n")
    progress_bar = tqdm(total=len(train_loader), leave=True, ncols=100)

    for batch_idx, (data, out_lengths, target, lengths) in enumerate(train_loader, start=1):

        optimizer.zero_grad()

        chunks += data.shape[0]

        data = data.to(device)
        target = target.to(device)

        # forward pass
        log_probs = model(data)

        # calculate loss
        loss = criterion(log_probs.transpose(0, 1), target, out_lengths / model.stride, lengths)

        # backward pass
        loss.backward()

        # update weights
        optimizer.step()
        progress_bar.refresh()
        progress_bar.update(1)
        progress_bar.set_description("Loss: " + str(loss.item()))
        sys.stderr.flush()        

    progress_bar.close()

    return loss.item(), time.perf_counter() - t0


def test(model, device, test_loader):

    model.eval()
    test_loss = 0
    predictions = []
    prediction_lengths = []

    with torch.no_grad():
        for batch_idx, (data, out_lengths, target, lengths) in enumerate(test_loader, start=1):
            data, target = data.to(device), target.to(device)
 
            # forward pass
            log_probs = model(data)
 
            # calculate loss
            test_loss += criterion(log_probs.transpose(1, 0), target, out_lengths / model.stride, lengths)

            # accumulate output probabilities
            predictions.append(torch.exp(log_probs).cpu())
            prediction_lengths.append(out_lengths / model.stride)

    predictions = np.concatenate(predictions)
    lengths = np.concatenate(prediction_lengths)

    # convert probabilities to sequences
    references = [decode_ref(target, model.alphabet) for target in test_loader.dataset.targets]
    sequences = [decode(post[:n], model.alphabet) for post, n in zip(predictions, lengths)]

    # align predicted sequences with true sequences and calculate accuracy
    if all(map(len, sequences)):
        accuracies = list(starmap(accuracy, zip(references, sequences)))
    else:
        accuracies = [0]

    # measure average accuracies over entire set of validation chunks
    mean = np.mean(accuracies)
    median = np.median(accuracies)

    return test_loss.item() / batch_idx, mean, median

#### Main training loop
When this cell is run we begin training the network.

In [0]:
#@title Set experiment name
experiment_name = 'bonito_training_321' #@param {type:"string"}

In [0]:
# mount users drive to save data
gdrive.mount('/content/drive', force_remount=True)

# prevent overwriting of data
workdir = os.path.join(model_savepath, experiment_name)
if os.path.isdir(workdir):
    raise IOError('{} already exists. Select an alternative model_savepath.'.format(workdir))
os.makedirs(workdir)

# data generators
train_dataset = ChunkDataSet(chunks[:split], chunk_lengths[:split],
                             targets[:split], target_lengths[:split])
test_dataset = ChunkDataSet(chunks[split:], chunk_lengths[split:],
                            targets[split:], target_lengths[split:])

# data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,
                         num_workers=4, pin_memory=True)

# load bonito model
model = Model(quartznet_config)
model.to(device)
model.train()

# set optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), amsgrad=True, lr=learning_rate)
schedular = CosineAnnealingLR(optimizer, epochs * len(train_loader))

# report loss every 
interval = 500 / num_chunks
log_interval = np.floor(len(train_dataset) / batch_size * interval)

exp_config = os.path.join(workdir, "experimental.log")
with open(exp_config, 'a') as c:
    c.write('Num training chunks: {}'.format(num_chunks) + '\n')
    c.write('learning rate: {}'.format(learning_rate) + '\n')
    c.write('random seed: {}'.format(random_seed) + '\n')
    c.write('epochs: {}'.format(epochs) + '\n')
    c.write('batch_size: {}'.format(batch_size) + '\n')
    c.write('train proportion: {}'.format(train_proportion) + '\n')
    c.write('dropout: {}'.format(dropout) + '\n')

# DataFrame to store training logging information
training_results = pd.DataFrame()

for epoch in range(1, epochs + 1):

    train_loss, duration = train(log_interval, model, device,
                                 train_loader, optimizer, epoch)
    
    test_loss, mean, median = test(model, device, test_loader)

    # collate training and validation metrics
    epoch_result = pd.DataFrame(
        {'time':[datetime.today()],
         'duration':[int(duration)],
         'epoch':[epoch],
         'train_loss':[train_loss],
         'validation_loss':[test_loss], 
         'validation_mean':[mean],
         'validation_median':[median]})
    
    # save model weights
    weights_path = os.path.join(workdir, "weights_%s.tar" % epoch)
    torch.save(model.state_dict(), weights_path)

    # update log file
    log_path = os.path.join(workdir, "training.log")
    epoch_result.to_csv(log_path, mode='a', sep='\t', index=False)

    display(epoch_result)
    training_results = training_results.append(epoch_result)

    schedular.step()

display(training_results)

Complete training log will be displayed upon training completion.

Partial logs are saved to the model_savepath.